In [7]:
import pandas as pd
import numpy as np

- 데이터 셋 호출.

In [8]:
train = pd.read_csv('train.csv', encoding = 'utf-8')
test = pd.read_csv('test.csv',encoding = 'utf-8')
sub = pd.read_csv('sample_submission.csv', encoding = 'utf-8')
age = pd.read_csv('age_gender_info.csv', encoding = 'utf-8')

In [9]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
      dtype='object')

- target
    - 등록차량수

- feature 
    - category
        - 단지코드
        - 임대건물구분
        - 지역
        - 공급유형
        - 자격유형
        
    - numeric
        - 총세대수
        - 전용면적
        - 전용면적별세대수
        - 공가수
        - 임대보증금
        - 임대료
        - 도보 10분거리 내 지하철역 수(환승노선 수 반영)
        - 도보 10분거리 내 버스정류장 수
        - 단지내주차면수

In [10]:
df_train = train.copy()
df_test = test.copy()

In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

In [12]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          1022 non-null   object 
 1   총세대수                          1022 non-null   int64  
 2   임대건물구분                        1022 non-null   object 
 3   지역                            1022 non-null   object 
 4   공급유형                          1022 non-null   object 
 5   전용면적                          1022 non-null   float64
 6   전용면적별세대수                      1022 non-null   int64  
 7   공가수                           1022 non-null   float64
 8   자격유형                          1020 non-null   object 
 9   임대보증금                         842 non-null    object 
 10  임대료                           842 non-null    object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  980 non-null    float64
 12  도보 10분거리 내 버스정류장 수            1022 non-null   float64
 13  단지내

- null 값 처리
- type 변경

- 임대료, 임대보증금에는 0값대신 평균값 대입

In [13]:
cols = ['도보 10분거리 내 지하철역 수(환승노선 수 반영)','도보 10분거리 내 버스정류장 수','임대료','임대보증금']
df_train[cols] = df_train[cols].fillna(0)
df_test[cols] = df_test[cols].fillna(0)

cols = ['임대료','임대보증금']
for col in cols:
    df_train[col] = df_train[col].replace('-',0).astype('int64')
    df_train[col] = df_train[col].replace(0,np.round(df_train[col].mean(),0))
    df_test[col] = df_test[col].replace('-',0).astype('int64')
    df_test[col] = df_test[col].replace(0,np.round(df_test[col].mean(),0))

- 파생 변수 생성
    - 도보 10분거리 내 정류장(역) 수 : 지하철역 수 + 버스정류장 수

- 변수 제거 
    - delete 
        - 공급유형
        - 신분
        - 도보 10분거리 내 지하철역 수(환승노선 수 반영)
        - 도보 10분거리 내 버스정류장 수


In [14]:
cols = ['도보 10분거리 내 지하철역 수(환승노선 수 반영)','도보 10분거리 내 버스정류장 수']
df_train['도보 10분거리 내 정류장(역) 수'] = df_train.loc[:,cols].sum(axis = 1)
df_test['도보 10분거리 내 정류장(역) 수'] = df_test.loc[:,cols].sum(axis = 1)


df_train['도보 10분거리 내 정류장(역) 수'] = df_train.loc[:,cols].sum(axis = 1)
df_test['도보 10분거리 내 정류장(역) 수'] = df_test.loc[:,cols].sum(axis = 1)

cols.extend(['자격유형','공급유형'])
for col in cols:
    del df_train[col]
    del df_test[col]

In [15]:
df_train.shape, df_test.shape

((2952, 12), (1022, 11))

- dictinary 에 train set의 label input
    - 서울특별시가 test셋에는 없다.

In [16]:
# length = np.array(range(0,16))
# label = dict(zip(length,lr.classes_))
# label

In [17]:
zero = pd.DataFrame(np.zeros(len(df_test)),columns = ['서울특별시'])

In [18]:
zero = pd.DataFrame(np.zeros(len(df_test)),columns = ['서울특별시'])
df_train = pd.concat([df_train,pd.get_dummies(df_train['지역']),pd.get_dummies(df_train['임대건물구분'])], axis = 1)
df_test = pd.concat([df_test,pd.get_dummies(df_test['지역']),zero ,pd.get_dummies(df_test['임대건물구분'])], axis = 1)
drop_cols = ['지역','임대건물구분']


for col in drop_cols:
    del df_train[col]
    del df_test[col]

In [19]:
# for col in ['임대보증금','임대료']:
#     df_train[col] = pd.Series(map(np.log,df_train[col]))
#     df_test[col] = pd.Series(map(np.log,df_test[col]))

In [20]:
df_train

,단지코드,총세대수,전용면적,전용면적별세대수,공가수,임대보증금,임대료,단지내주차면수,등록차량수,도보 10분거리 내 정류장(역) 수,...,서울특별시,세종특별자치시,울산광역시,전라남도,전라북도,제주특별자치도,충청남도,충청북도,상가,아파트
0,C2483,900,39.72,134,38.0,15667000.0,103680.0,1425.0,1015.0,3.0,...,0,0,0,0,0,0,0,0,0,1
1,C2483,900,39.72,15,38.0,15667000.0,103680.0,1425.0,1015.0,3.0,...,0,0,0,0,0,0,0,0,0,1
2,C2483,900,51.93,385,38.0,27304000.0,184330.0,1425.0,1015.0,3.0,...,0,0,0,0,0,0,0,0,0,1
3,C2483,900,51.93,15,38.0,27304000.0,184330.0,1425.0,1015.0,3.0,...,0,0,0,0,0,0,0,0,0,1
4,C2483,900,51.93,41,38.0,27304000.0,184330.0,1425.0,1015.0,3.0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,49.20,19,7.0,11346000.0,116090.0,166.0,146.0,1.0,...,0,0,0,0,0,0,0,0,0,1
2948,C2532,239,51.08,34,7.0,14005000.0,142310.0,166.0,146.0,1.0,...,0,0,0,0,0,0,0,0,0,1
2949,C2532,239,51.73,34,7.0,14005000.0,142310.0,166.0,146.0,1.0,...,0,0,0,0,0,0,0,0,0,1
2950,C2532,239,51.96,114,7.0,14005000.0,142310.0,166.0,146.0,1.0,...,0,0,0,0,0,0,0,0,0,1


In [21]:
df_train.shape, df_test.shape

((2952, 28), (1022, 27))

In [22]:
df_train = np.round(df_train.groupby('단지코드').mean(),0)
df_test = np.round(df_test.groupby('단지코드').mean(),0)


In [23]:
df_train.shape, df_test.shape

((423, 27), (150, 26))

In [24]:
df_train.columns

Index(['총세대수', '전용면적', '전용면적별세대수', '공가수', '임대보증금', '임대료', '단지내주차면수', '등록차량수',
       '도보 10분거리 내 정류장(역) 수', '강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시',
       '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '전라남도', '전라북도',
       '제주특별자치도', '충청남도', '충청북도', '상가', '아파트'],
      dtype='object')

In [25]:
df_test.columns

Index(['총세대수', '전용면적', '전용면적별세대수', '공가수', '임대보증금', '임대료', '단지내주차면수',
       '도보 10분거리 내 정류장(역) 수', '강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시',
       '대전광역시', '부산광역시', '세종특별자치시', '울산광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도',
       '충청북도', '서울특별시', '상가', '아파트'],
      dtype='object')

In [26]:
feature_cols = df_train.columns.difference(['등록차량수'])
x_train = df_train[feature_cols]
y_train = df_train['등록차량수']
x_test = df_test[feature_cols]
x_train.shape, y_train.shape, x_test.shape

((423, 26), (423,), (150, 26))

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection import GridSearchCV

In [28]:
lr = LinearRegression().fit(x_train, y_train)
lr.score(x_train,y_train)

0.8103554835192117

In [41]:
rf_grid = [
        {'n_estimators': [3, 10, 30], 
        'max_features': [2, 4, 6, 8]}
    ]


rf = RandomForestRegressor()
fit_rf = GridSearchCV(rf, param_grid =  rf_grid, cv=5)
fit_rf.fit(x_train, y_train)
fit_rf.best_params_

{'max_features': 6, 'n_estimators': 30}

In [38]:
gbm_grid = [
    {'n_estimators' : [100, 300, 500 ],
     'learning_rate' : [0.05, 0.1]
    }
]
gbm = GradientBoostingRegressor()
fit_gbm = GridSearchCV(gbm, param_grid=gbm_grid, cv=5, verbose=1, n_jobs=-1).fit(x_train, y_train)
fit_gbm.best_params_

Fitting 5 folds for each of 6 candidates, totalling 30 fits


{'learning_rate': 0.05, 'n_estimators': 100}

In [39]:
lgbm_grid = [
    {'n_estimators' : [100, 300, 500 ],
     'learning_rate' : [0.05, 0.1]
    }
]

lgbm = lightgbm.LGBMRegressor()
fit_lgbm = GridSearchCV(lgbm, param_grid = lgbm_grid, cv = 5, verbose = 1, n_jobs = -1).fit(x_train, y_train)
fit_lgbm.best_params_

Fitting 5 folds for each of 6 candidates, totalling 30 fits


{'learning_rate': 0.05, 'n_estimators': 100}

In [40]:
xgb_grid = [
    {'n_estimators' : [100, 300, 500 ],
     'learning_rate' : [0.05, 0.1]
    }
]

xgb = xgboost.XGBRegressor()
fit_xgb = GridSearchCV(xgb, param_grid = xgb_grid, cv = 5, verbose = 1, n_jobs = -1).fit(x_train, y_train)
fit_xgb.best_params_

Fitting 5 folds for each of 6 candidates, totalling 30 fits


{'learning_rate': 0.1, 'n_estimators': 100}

In [ ]:
rf = RandomForestRegressor(max_features = 8, n_estimators = 30, criterion= 'mse').fit(x_train,y_train)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor
import xgboost
import lightgbm

In [43]:
MLA = [
    # Linear model
    LinearRegression(),
    
    # Bagging 
    BaggingRegressor(),

    # ensemble
    # RandomForestRegressor(max_features =  6, n_estimators = 30),
    GradientBoostingRegressor(learning_rate = 0.05, n_estimators = 100),
    AdaBoostRegressor(),


    # xgboost
    xgboost.XGBRegressor(learning_rate = 0.1, n_estimators = 100),

    # LGBM
    lightgbm.LGBMRegressor(learning_rate =  0.05, n_estimators =  100)
]

In [47]:
rf = RandomForestRegressor(max_features= 6, n_estimators= 30)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)

In [45]:
rf.fit(x_train, y_train)
result = rf.predict(x_test)
for model in MLA:
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    result += y_pred
y_pred = result / 7



In [50]:
sub = sub.sort_values('code')
sub['num'] = y_pred
sub.to_csv('result.csv',index = False)

In [49]:
 lgbm = lightgbm.LGBMRegressor(learning_rate =  0.05, n_estimators =  100)
 lgbm.fit(x_train, y_train)
 y_pred = lgbm.predict(x_test)